# Making HTTP Requests

- The `requests` library simplifies HTTP interactions by abstracting raw HTTP details, making it ideal for DevOps automation tasks.
- It allows you to query web services, trigger CI/CD builds, manage cloud resources, and integrate with APIs like GitHub in a straightforward way.
- In this notebook, we'll demonstrate installing `requests`, performing GET and POST requests, inspecting response data, and customizing requests with parameters and headers.
- The `requests` library is a third-party package and must be installed in your active virtual environment.
- Use `pip install requests==2.32.2` to add it to your project (pinning the version here so that we all work with the same version, but in other projects you can omit the version to install the latest), and consider pinning its version in `requirements.txt`.

In [50]:
GITHUB_ENDPOINT = "https://api.github.com"
HTTPBIN_ENDPOINT = "https://httpbin.org"

## Making GET Requests with `requests.get()`

- The GET method retrieves data from a specified URL; it’s the most common HTTP request type.
- Key parameters include `url`, optional `params` for query strings, `headers` for custom HTTP headers, and `timeout` to avoid hanging requests.
- The returned `Response` object provides `.status_code`, `.headers`, `.text`, `.content`, and `.json()`, plus `.raise_for_status()` to handle HTTP errors.

In [53]:
import requests
import json

response = requests.get("https://api.github.com", timeout=10)

"""
print(f"Status code: {response.status_code}")
print(f"Content-type: {response.headers['content-type']}")
print(".text attribute:")
print(response.text)
print("\n.content attribute:")
print(response.content)
print("\n.json() attribute:")
print(response.json())

"""

data = response.json()
print("Avaiable endpoints:")
print(json.dumps(data, indent=2))

Avaiable endpoints:
{
  "current_user_url": "https://api.github.com/user",
  "current_user_authorizations_html_url": "https://github.com/settings/connections/applications{/client_id}",
  "authorizations_url": "https://api.github.com/authorizations",
  "code_search_url": "https://api.github.com/search/code?q={query}{&page,per_page,sort,order}",
  "commit_search_url": "https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}",
  "emails_url": "https://api.github.com/user/emails",
  "emojis_url": "https://api.github.com/emojis",
  "events_url": "https://api.github.com/events",
  "feeds_url": "https://api.github.com/feeds",
  "followers_url": "https://api.github.com/user/followers",
  "following_url": "https://api.github.com/user/following{/target}",
  "gists_url": "https://api.github.com/gists{/gist_id}",
  "hub_url": "https://api.github.com/hub",
  "issue_search_url": "https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}",
  "issues_url": "https://a

## Passing URL Parameters with `params`

- Query parameters are passed as a dictionary to the `params` argument, and `requests` handles URL-encoding automatically.
- This makes it easy to filter, sort, or paginate API results without manually constructing the query string.
- You can inspect the final URL via `response.url` to confirm your parameters were applied correctly.

In [57]:
import requests
import json 

search_url = f"{GITHUB_ENDPOINT}/search/repositories"
query_params = {
    "q": "python devops", 
    "sort": "stars", 
    "order": "desc",
    "per_page": 5
    }

response = requests.get(search_url, params=query_params, timeout=10)
response.raise_for_status()  # 4xx or 5xx errors will be raised

print(f"Requested URL: {response.url}")
result = response.json()

print(f"Found {result['total_count']} repositories. Top 5")
for repo in result.get("items", []):
    print(f"- {repo['name']}:(Stars: {repo['stargazers_count']})")

print(json.dumps(result.get("items", [])[0], indent=2))    

Requested URL: https://api.github.com/search/repositories?q=python+devops&sort=stars&order=desc&per_page=5
Found 7882 repositories. Top 5
- devops-exercises:(Stars: 79449)
- sentry:(Stars: 42262)
- cli:(Stars: 36818)
- jumpserver:(Stars: 28880)
- DevOps-Roadmap:(Stars: 17343)
{
  "id": 212639071,
  "node_id": "MDEwOlJlcG9zaXRvcnkyMTI2MzkwNzE=",
  "name": "devops-exercises",
  "full_name": "bregman-arie/devops-exercises",
  "private": false,
  "owner": {
    "login": "bregman-arie",
    "id": 10349437,
    "node_id": "MDQ6VXNlcjEwMzQ5NDM3",
    "avatar_url": "https://avatars.githubusercontent.com/u/10349437?v=4",
    "gravatar_id": "",
    "url": "https://api.github.com/users/bregman-arie",
    "html_url": "https://github.com/bregman-arie",
    "followers_url": "https://api.github.com/users/bregman-arie/followers",
    "following_url": "https://api.github.com/users/bregman-arie/following{/other_user}",
    "gists_url": "https://api.github.com/users/bregman-arie/gists{/gist_id}",
    "st

## Making POST Requests with `requests.post()`

- Use `requests.post()` to send data to a server, choosing between `data=` for form-encoded bodies or `json=` for JSON payloads.
- Providing a dictionary to `json=` automatically serializes it and sets `Content-Type: application/json`.
- The response can be inspected similarly to GET responses, using `.status_code`, `.json()`, and error handling.

In [59]:
import requests
import json

post_echo_url = f"{HTTPBIN_ENDPOINT}/post"

payload = {
    "script_name": "devops_automation",
    "action": "trigger_deployment",
    "environment": "staging",
    "version": "v1.5.0"
}

response = requests.post(post_echo_url, json=payload, timeout=10)
response.raise_for_status()  # 4xx or 5xx errors will be raised

print(json.dumps(response.json(), indent=2))

{
  "args": {},
  "data": "{\"script_name\": \"devops_automation\", \"action\": \"trigger_deployment\", \"environment\": \"staging\", \"version\": \"v1.5.0\"}",
  "files": {},
  "form": {},
  "headers": {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Content-Length": "115",
    "Content-Type": "application/json",
    "Host": "httpbin.org",
    "User-Agent": "python-requests/2.32.3",
    "X-Amzn-Trace-Id": "Root=1-68f7b8a6-0e0d72f838614f3178246dfd"
  },
  "json": {
    "action": "trigger_deployment",
    "environment": "staging",
    "script_name": "devops_automation",
    "version": "v1.5.0"
  },
  "origin": "178.27.160.57",
  "url": "https://httpbin.org/post"
}


## Common Pitfalls & How to Avoid Them

- **Not setting timeouts** can cause scripts to hang indefinitely; always include a `timeout` value.
- **Ignoring HTTP errors** means you might assume success when a request failed; use `response.raise_for_status()`.
- **Using `data=` instead of `json=`** sends form-encoded data, which may be rejected by modern APIs expecting JSON.
- **Hardcoding secrets** in code is insecure; use environment variables (e.g., via `python-dotenv`) and pass them in headers.